# Markov Network fundamentals

## Markov Network Introduction
In the previous note, the Bayesian Network is introduced. However, in practice, there are many cases of CPD that Bayesian Network cannot describle (e.g. The misconception example in Learning Bayesian Network, the dinner problem in this book). In all of these cases, the interaction between variables are bi-directional. Therefore, an __undirected__ network is a better description for these interaction. A Markov Network is an __undirected__ network encoding the bi-directional probabilistic relationship between variables. Instead of using CPD, in Markov Network they use a more symmetric representation called _factor_ $\phi$, which basically represent __how likely it is for some states of a variable to _agree_ with the states of other variables__.

## Parameterizing a Markov network - factor
Definition: A factor $\phi$ of a set of random variables X is defined as a function mapping values of X to some real number R:
$$ \phi(X):Val(X) \rightarrow R $$
Unlike CPD, there is no restriction to the value of $\phi$ (maybe except possitivity). In this setting, the higher the value of $\phi(a^0,b^0)$ compare to $\phi(a^1, b^0)$ means that $a^0$ and $b^0$ are more likely to happend (more compatible) compare to the other combination.

In ```pgmpy```, we can define factors as follow:

In [1]:
# Import the module
from pgmpy.factors import Factor

In [4]:
# Each factor is represented by its scope (scope is set of random variable)
phi = Factor(['A', 'B'], [2,2], [1000, 1, 5, 100])

In [5]:
print(phi)

+-----+-----+------------+
| A   | B   |   phi(A,B) |
|-----+-----+------------|
| A_0 | B_0 |  1000.0000 |
| A_0 | B_1 |     1.0000 |
| A_1 | B_0 |     5.0000 |
| A_1 | B_1 |   100.0000 |
+-----+-----+------------+


## Factor operations
There are many mathematical operations on factors. This is few major ones: marginalization, reduction, product (* important *)

In [10]:
# Marginalize B: Sum up all factors, reduce the scope X to X' = X - B 
phi_marginalized = phi.marginalize(['B'], inplace=False)

In [11]:
phi_marginalized.scope()

['A']

In [12]:
print(phi_marginalized)

+-----+-----------+
| A   |    phi(A) |
|-----+-----------|
| A_0 | 1001.0000 |
| A_1 |  105.0000 |
+-----+-----------+


In [13]:
# The argument 'inplace' is to indicate if user wants a new Factor returned 
# or modify the original factor
phi.marginalize(['A'])

In [14]:
print(phi)

+-----+-----------+
| B   |    phi(B) |
|-----+-----------|
| B_0 | 1005.0000 |
| B_1 |  101.0000 |
+-----+-----------+


In [20]:
# Reduction (to a context Z) is to remove all entries of scope X that have Z. 
# This operation reduce the scope to X - Z, and \phi no longer depends on Z.
phi = Factor(['A', 'B'], [2,2], [1000, 1, 5, 100])
phi_reduced = phi.reduce([('B', 0)], inplace=False)

In [21]:
print(phi_reduced)

+-----+-----------+
| A   |    phi(A) |
|-----+-----------|
| A_0 | 1000.0000 |
| A_1 |    5.0000 |
+-----+-----------+


In [22]:
phi_reduced.scope()

['A']

In [23]:
phi_reduced_a1 = phi.reduce([('A', 1)], inplace=False)

In [24]:
print(phi_reduced_a1)

+-----+----------+
| B   |   phi(B) |
|-----+----------|
| B_0 |   5.0000 |
| B_1 | 100.0000 |
+-----+----------+


In [25]:
print(phi)

+-----+-----+------------+
| A   | B   |   phi(A,B) |
|-----+-----+------------|
| A_0 | B_0 |  1000.0000 |
| A_0 | B_1 |     1.0000 |
| A_1 | B_0 |     5.0000 |
| A_1 | B_1 |   100.0000 |
+-----+-----+------------+


In [34]:
# The most important operation is the product operation
phi1 = Factor(['a', 'b'], [2, 2], [1000, 1, 5, 100])
phi2 = Factor(['b', 'c'], [2, 3], [1, 100, 5, 200, 3, 1000])
phi3 = Factor(['c', 'd'], [3, 2], [2000, 3, 400, 200, 4, 1])
# With this expression, the last list the values list of the [3, 2] array. To remember
# this, image d is unit and has only 2 values and c is tens and has 3 values. Therefore
# the value listing will be: 00, 01, 10, 11, 20, 21 -> corresponds with the list.

In [35]:
# We can use the operator * for product
phi = phi1 * phi2

In [36]:
phi.scope()

['a', 'b', 'c']

In [37]:
print(phi)

+-----+-----+-----+--------------+
| a   | b   | c   |   phi(a,b,c) |
|-----+-----+-----+--------------|
| a_0 | b_0 | c_0 |    1000.0000 |
| a_0 | b_0 | c_1 |  100000.0000 |
| a_0 | b_0 | c_2 |    5000.0000 |
| a_0 | b_1 | c_0 |     200.0000 |
| a_0 | b_1 | c_1 |       3.0000 |
| a_0 | b_1 | c_2 |    1000.0000 |
| a_1 | b_0 | c_0 |       5.0000 |
| a_1 | b_0 | c_1 |     500.0000 |
| a_1 | b_0 | c_2 |      25.0000 |
| a_1 | b_1 | c_0 |   20000.0000 |
| a_1 | b_1 | c_1 |     300.0000 |
| a_1 | b_1 | c_2 |  100000.0000 |
+-----+-----+-----+--------------+


In [38]:
# The dimensionality between same variable ['b', 'c'] and ['c', 'd'] -> 'c' of both must
# have same cardinality
phi = phi2 * phi3

In [39]:
print(phi)

+-----+-----+-----+--------------+
| b   | c   | d   |   phi(b,c,d) |
|-----+-----+-----+--------------|
| b_0 | c_0 | d_0 |    2000.0000 |
| b_0 | c_0 | d_1 |       3.0000 |
| b_0 | c_1 | d_0 |   40000.0000 |
| b_0 | c_1 | d_1 |   20000.0000 |
| b_0 | c_2 | d_0 |      20.0000 |
| b_0 | c_2 | d_1 |       5.0000 |
| b_1 | c_0 | d_0 |  400000.0000 |
| b_1 | c_0 | d_1 |     600.0000 |
| b_1 | c_1 | d_0 |    1200.0000 |
| b_1 | c_1 | d_1 |     600.0000 |
| b_1 | c_2 | d_0 |    4000.0000 |
| b_1 | c_2 | d_1 |    1000.0000 |
+-----+-----+-----+--------------+


In [42]:
# PGMPY will automatically concatinate matching variable names. If there is no matching 
# variable names, PGMPY will create full table.
phi = phi1 * phi3 # also phi_new = phi.product(phi1, phi2)

In [43]:
print(phi)

+-----+-----+-----+-----+----------------+
| a   | b   | c   | d   |   phi(a,b,c,d) |
|-----+-----+-----+-----+----------------|
| a_0 | b_0 | c_0 | d_0 |   2000000.0000 |
| a_0 | b_0 | c_0 | d_1 |      3000.0000 |
| a_0 | b_0 | c_1 | d_0 |    400000.0000 |
| a_0 | b_0 | c_1 | d_1 |    200000.0000 |
| a_0 | b_0 | c_2 | d_0 |      4000.0000 |
| a_0 | b_0 | c_2 | d_1 |      1000.0000 |
| a_0 | b_1 | c_0 | d_0 |      2000.0000 |
| a_0 | b_1 | c_0 | d_1 |         3.0000 |
| a_0 | b_1 | c_1 | d_0 |       400.0000 |
| a_0 | b_1 | c_1 | d_1 |       200.0000 |
| a_0 | b_1 | c_2 | d_0 |         4.0000 |
| a_0 | b_1 | c_2 | d_1 |         1.0000 |
| a_1 | b_0 | c_0 | d_0 |     10000.0000 |
| a_1 | b_0 | c_0 | d_1 |        15.0000 |
| a_1 | b_0 | c_1 | d_0 |      2000.0000 |
| a_1 | b_0 | c_1 | d_1 |      1000.0000 |
| a_1 | b_0 | c_2 | d_0 |        20.0000 |
| a_1 | b_0 | c_2 | d_1 |         5.0000 |
| a_1 | b_1 | c_0 | d_0 |    200000.0000 |
| a_1 | b_1 | c_0 | d_1 |       300.0000 |
| a_1 | b_1

## Gibbs distributions and Markov Networks